<a href="https://colab.research.google.com/github/M-Kavya-2005/Project-1/blob/main/NASA_Near_Earth_Object_(NEO)_Tracking_%26_Insights_using_Public_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import requests

API_KEY = "yu1m2WriCMVAFWHHqKufh4t6nj63D9SFUM7xQhrq"

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_KEY}"

response = requests.get(url)

response


<Response [200]>

In [ ]:
data = response.json()
details = data['near_earth_objects']
details['2024-01-01'][0]

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3724393?api_key=yu1m2WriCMVAFWHHqKufh4t6nj63D9SFUM7xQhrq'},
 'id': '3724393',
 'neo_reference_id': '3724393',
 'name': '(2015 OD22)',
 'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3724393',
 'absolute_magnitude_h': 21.21,
 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.152249185,
   'estimated_diameter_max': 0.3404395273},
  'meters': {'estimated_diameter_min': 152.249185036,
   'estimated_diameter_max': 340.4395272595},
  'miles': {'estimated_diameter_min': 0.0946032284,
   'estimated_diameter_max': 0.2115392495},
  'feet': {'estimated_diameter_min': 499.5052162336,
   'estimated_diameter_max': 1116.9276186141}},
 'is_potentially_hazardous_asteroid': False,
 'close_approach_data': [{'close_approach_date': '2024-01-01',
   'close_approach_date_full': '2024-Jan-01 11:58',
   'epoch_date_close_approach': 1704110280000,
   'relative_velocity': {'kilometers_per_second': '25.3931645932',
  

In [ ]:
from datetime import datetime
asteroids_data = []

target = 10000

while len(asteroids_data) < target:
    response = requests.get(url)
    data = response.json()
    details = data['near_earth_objects']
    for date,asteroids in details.items():
        for ast in asteroids:
            close_approach_list = ast.get('close_approach_data', [])
            approach = close_approach_list[0]
            asteroids_data.append(dict(
                    id=int(ast.get('id', 0)),
                    neoReferenceId=int(ast.get('neo_reference_id', 0)),
                    name=ast.get('name', ''),
                    absoluteMagnitudeH=ast.get('absolute_magnitude_h', 0.0),
                    estimatedMinDiameter=ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_min', 0.0),
                    estimatedMaxDiameter=ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_max', 0.0),
                    isPotentiallyHazardousAsteroid=ast.get('is_potentially_hazardous_asteroid', False),
                    closeApproachDate=datetime.strptime(approach.get('close_approach_date', '1900-01-01'), "%Y-%m-%d").date(),
                    relativeVelocity=float(approach.get('relative_velocity', {}).get('kilometers_per_hour', 0.0)),
                    astronomical=float(approach.get('miss_distance', {}).get('astronomical', 0.0)),
                    missDistance=float(approach.get('miss_distance', {}).get('kilometers', 0.0)),
                    missDistanceLunar=float(approach.get('miss_distance', {}).get('lunar', 0.0)),
                    orbitingBody=approach.get('orbiting_body', '')
                ))
            if len(asteroids_data) >= target:
                break
        if len(asteroids_data) >= target:
                break

    url = data['links']['next']

len(asteroids_data)

10000

In [ ]:
asteroids_data[0]

{'id': 2415949,
 'neoReferenceId': '2415949',
 'name': '415949 (2001 XY10)',
 'absoluteMagnitudeH': 19.37,
 'estimatedMinDiameter': 0.3552670883,
 'estimatedMaxDiameter': 0.7944013596,
 'isPotentiallyHazardousAsteroid': False,
 'closeApproachDate': datetime.date(2024, 1, 2),
 'relativeVelocity': 57205.8951204341,
 'astronomical': 0.3372535274,
 'missDistance': 50452409.349026635,
 'missDistanceLunar': 131.1916221586,
 'orbitingBody': 'Earth'}

In [ ]:
import sqlite3

conn = sqlite3.connect("asteroids.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS asteroids (
    id INTEGER PRIMARY KEY,
    name VARCHAR,
    absolute_magnitude_h FLOAT,
    estimated_min_diameter FLOAT,
    estimated_max_diameter FLOAT,
    is_potentially_hazardous BOOLEAN
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS close_approach (
    neo_reference_id INTEGER,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR,
    FOREIGN KEY (neo_reference_id) REFERENCES asteroids(id)
)
""")
conn.commit()

In [ ]:
insert_asteroid_sql = """
INSERT OR IGNORE INTO asteroids (
    id, name, absolute_magnitude_h,
    estimated_min_diameter, estimated_max_diameter,
    is_potentially_hazardous
) VALUES (?, ?, ?, ?, ?, ?)
"""

insert_close_approach_sql = """
INSERT INTO close_approach (
    neo_reference_id, close_approach_date, relative_velocity_kmph,
    astronomical, miss_distance_km, miss_distance_lunar, orbiting_body
) VALUES (?, ?, ?, ?, ?, ?, ?)
"""

for asteroid in asteroids_data:
    cursor.execute(insert_asteroid_sql, (
        asteroid['id'],
        asteroid['name'],
        asteroid['absoluteMagnitudeH'],
        asteroid['estimatedMinDiameter'],
        asteroid['estimatedMaxDiameter'],
        asteroid['isPotentiallyHazardousAsteroid']
    ))

    cursor.execute(insert_close_approach_sql, (
        asteroid['neoReferenceId'],
        asteroid['closeApproachDate'],
        asteroid['relativeVelocity'],
        asteroid['astronomical'],
        asteroid['missDistance'],
        asteroid['missDistanceLunar'],
        asteroid['orbitingBody']
    ))

conn.commit()

In [ ]:
import pandas as pd

conn = sqlite3.connect("asteroids.db")

1. Count how many times each asteroid has approached Earth

In [ ]:
approach_counts_df = pd.read_sql_query("""
SELECT neo_reference_id, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id
""", conn)

display(approach_counts_df)

,neo_reference_id,approach_count
0,2000887,1
1,2001685,1
2,2002063,1
3,2002340,1
4,2004034,1
...,...,...
7498,54533502,1
7499,54533907,1
7500,54536909,1
7501,54537502,1


2. Average velocity of each asteroid over multiple approaches

In [ ]:
avg_velocity_df = pd.read_sql_query("""SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
FROM close_approach
GROUP BY neo_reference_id""", conn)

display(avg_velocity_df)

,neo_reference_id,avg_velocity
0,2000887,29695.315943
1,2001685,58226.721077
2,2002063,30698.842978
3,2002340,93669.384688
4,2004034,43128.810179
...,...,...
7498,54533502,14929.830125
7499,54533907,37650.646735
7500,54536909,15849.661548
7501,54537502,65210.016542


3. Top 10 fastest asteroids

In [ ]:
fastest_asteroids_df = pd.read_sql_query("""SELECT neo_reference_id, MAX(relative_velocity_kmph) AS max_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY max_velocity DESC
LIMIT 10""", conn)

display(fastest_asteroids_df)

,neo_reference_id,max_velocity
0,54277079,190513.420686
1,54392072,173071.830081
2,3803884,161520.897925
3,3790060,158279.864701
4,3425295,154336.407297
5,3092220,152218.220518
6,3588900,151520.395439
7,54448599,150599.454097
8,54111253,150583.019419
9,3383901,149261.278634


4. Potentially hazardous asteroids with more than 3 approaches

In [ ]:
hazardous_asteroids_df = pd.read_sql_query("""SELECT a.name, ca.neo_reference_id, COUNT(*) AS approaches
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
WHERE a.is_potentially_hazardous = 1
GROUP BY ca.neo_reference_id
HAVING approaches > 3""", conn)

display(hazardous_asteroids_df)

,name,neo_reference_id,approaches
0,(2010 VD72),3551067,4


5. Month with the most asteroid approaches

In [ ]:
asteroid_approaches_df = pd.read_sql_query("""SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS count
FROM close_approach
GROUP BY month
ORDER BY count DESC
LIMIT 1""", conn)

display(asteroid_approaches_df)

,month,count
0,2024-10,908


6. Asteroid with the fastest ever approach speed

In [ ]:
approach_speed_df = pd.read_sql_query("""SELECT neo_reference_id, MAX(relative_velocity_kmph) AS max_speed
FROM close_approach""", conn)

display(approach_speed_df)

,neo_reference_id,max_speed
0,54277079,190513.420686


7. Sort asteroids by max estimated diameter

In [ ]:
estimated_diameter_df = pd.read_sql_query("""SELECT name, estimated_max_diameter
FROM asteroids
ORDER BY estimated_max_diameter DESC""", conn)

display(estimated_diameter_df)

,name,estimated_max_diameter
0,887 Alinda (A918 AA),10.233872
1,1685 Toro (1948 OA),8.280184
2,66146 (1998 TU3),7.980681
3,5660 (1974 MA),4.989304
4,533671 (2014 LJ21),3.926811
...,...,...
7498,(2011 CQ1),0.002260
7499,(2024 YW8),0.002208
7500,(2023 RS),0.002042
7501,(2024 RC42),0.001986


8. Asteroids whose closest approach is getting nearer over time

In [ ]:
closest_approach_df = pd.read_sql_query("""SELECT neo_reference_id, close_approach_date, miss_distance_km
FROM close_approach
ORDER BY neo_reference_id, close_approach_date""", conn)

display(closest_approach_df)

,neo_reference_id,close_approach_date,miss_distance_km
0,2000887,2025-01-08,1.229661e+07
1,2001685,2024-01-20,1.990593e+07
2,2002063,2024-03-31,1.795253e+07
3,2002340,2024-11-02,5.948137e+07
4,2004034,2025-04-01,2.345060e+07
...,...,...,...
9995,54533502,2024-05-12,6.763441e+06
9996,54533907,2024-11-29,1.386360e+07
9997,54536909,2024-12-05,9.703563e+06
9998,54537502,2024-07-21,1.167117e+07


9. Name, date, and miss distance of closest approach

In [ ]:
miss_distance_df = pd.read_sql_query("""SELECT a.name, ca.close_approach_date, MIN(ca.miss_distance_km) AS closest_distance
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
GROUP BY a.name""", conn)

display(miss_distance_df)

,name,close_approach_date,closest_distance
0,(1989 AZ),2024-09-15,2.807433e+07
1,(1993 KA),2024-03-17,4.252127e+07
2,(1994 GL),2024-04-24,1.719310e+07
3,(1995 FO),2024-02-16,1.991364e+07
4,(1996 BA1),2024-03-01,5.220323e+07
...,...,...,...
7498,85953 (1999 FK21),2025-04-08,1.207825e+07
7499,86667 (2000 FO10),2024-05-15,2.953679e+07
7500,88188 (2000 XH44),2024-04-05,6.956269e+07
7501,887 Alinda (A918 AA),2025-01-08,1.229661e+07


10. Asteroids with velocity > 50,000 km/h

In [ ]:
asteroids_velocity_df = pd.read_sql_query("""SELECT DISTINCT a.name, ca.relative_velocity_kmph
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
WHERE ca.relative_velocity_kmph > 50000""", conn)

display(asteroids_velocity_df)

,name,relative_velocity_kmph
0,415949 (2001 XY10),57205.895120
1,(2005 YQ96),56413.014352
2,(2017 YD8),68377.113479
3,(2019 KK5),74999.269000
4,613286 (2005 YQ96),56413.042562
...,...,...
3108,(2025 HO4),71701.114559
3109,(2002 JQ100),97122.588494
3110,(2013 GR38),63180.000629
3111,(2014 YE1),102196.542405


11. Number of approaches per month

In [ ]:
numof_approaches_df = pd.read_sql_query("""SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS count
FROM close_approach
GROUP BY month""", conn)

display(numof_approaches_df)

,month,count
0,2024-01,628
1,2024-02,622
2,2024-03,637
3,2024-04,713
4,2024-05,563
5,2024-06,493
6,2024-07,469
7,2024-08,591
8,2024-09,841
9,2024-10,908


12. Asteroid with highest brightness (lowest magnitude)

In [ ]:
highest_brightness_df = pd.read_sql_query("""SELECT name, MIN(absolute_magnitude_h) AS brightest
FROM asteroids""", conn)

display(highest_brightness_df)

,name,brightest
0,887 Alinda (A918 AA),13.82


13. Count hazardous vs non-hazardous

In [ ]:
hazardous_nonhazardous_df = pd.read_sql_query("""SELECT is_potentially_hazardous, COUNT(*) AS count
FROM asteroids
GROUP BY is_potentially_hazardous""", conn)

display(hazardous_nonhazardous_df)

,is_potentially_hazardous,count
0,0,7111
1,1,392


14. Asteroids that passed closer than the Moon (<1 LD)

In [ ]:
asteroids_closer_df = pd.read_sql_query("""SELECT a.name, ca.close_approach_date, ca.miss_distance_lunar
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
WHERE ca.miss_distance_lunar < 1""", conn)

display(asteroids_closer_df)

,name,close_approach_date,miss_distance_lunar
0,(2024 AD),2024-01-04,0.631031
1,(2024 BA16),2024-01-15,0.364068
2,(2024 AZ3),2024-01-14,0.647173
3,(2024 AG4),2024-01-11,0.885273
4,(2024 AM4),2024-01-11,0.310675
...,...,...,...
231,(2025 GY),2025-04-06,0.900123
232,(2025 HA1),2025-04-20,0.302414
233,(2025 GS1),2025-04-15,0.098785
234,(2025 HH),2025-04-17,0.072433


15. Asteroids that came within 0.05 AU

In [ ]:
asteroids_came_df = pd.read_sql_query("""SELECT a.name, ca.close_approach_date, ca.astronomical
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
WHERE ca.astronomical < 0.05""", conn)

display(asteroids_came_df)

,name,close_approach_date,astronomical
0,(2019 KK5),2024-01-02,0.026097
1,(2023 YR),2024-01-02,0.011564
2,(2024 AA),2024-01-02,0.004081
3,(2024 AQ1),2024-01-02,0.043829
4,(2023 YR1),2024-01-01,0.016608
...,...,...,...
2535,(2025 HN),2025-04-23,0.013380
2536,(2025 HH1),2025-04-23,0.035292
2537,(2025 HX3),2025-04-23,0.007650
2538,(2025 HO4),2025-04-23,0.030683


16. Find the average brightness (magnitude) of hazardous asteroids

In [ ]:
avg_hazardous_brightness_df = pd.read_sql_query("""
SELECT AVG(absolute_magnitude_h) AS average_hazardous_brightness
FROM asteroids
WHERE is_potentially_hazardous = 1
""", conn)

display(avg_hazardous_brightness_df)

,average_hazardous_brightness
0,20.64273


17. List all approaches that happened in the year 2024

In [ ]:
approaches_2024_df = pd.read_sql_query("""
SELECT a.name, ca.close_approach_date, ca.miss_distance_km
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
WHERE strftime('%Y', ca.close_approach_date) = '2024'
ORDER BY ca.close_approach_date
""", conn)

display(approaches_2024_df)

,name,close_approach_date,miss_distance_km
0,(2015 OD22),2024-01-01,6.541738e+07
1,(2018 BA),2024-01-01,5.925668e+07
2,(2020 BC),2024-01-01,2.507066e+07
3,(2021 SG2),2024-01-01,7.179448e+07
4,(2022 AO2),2024-01-01,5.504957e+07
...,...,...,...
7828,(2025 AN1),2024-12-31,4.196498e+06
7829,(2025 AQ1),2024-12-31,1.961758e+07
7830,(2025 AB3),2024-12-31,2.028429e+07
7831,(2025 AA4),2024-12-31,6.903056e+05


18. Find which asteroid has approached Earth most frequently

In [ ]:
most_frequent_asteroid_df = pd.read_sql_query("""
SELECT a.name, COUNT(*) AS approach_count
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
GROUP BY a.name
ORDER BY approach_count DESC
LIMIT 1
""", conn)

display(most_frequent_asteroid_df)

,name,approach_count
0,(2025 FJ22),5


19. Find average miss distance of non-hazardous asteroids

In [ ]:
avg_non_hazardous_miss_distance_df = pd.read_sql_query("""
SELECT AVG(ca.miss_distance_km) AS average_non_hazardous_miss_distance
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
WHERE a.is_potentially_hazardous = 0
""", conn)
display(avg_non_hazardous_miss_distance_df)

,average_non_hazardous_miss_distance
0,2.847459e+07


 20. List all unique orbiting bodies

In [ ]:
orbiting_bodies_df = pd.read_sql_query("""
SELECT DISTINCT orbiting_body
FROM close_approach
""", conn)

display(orbiting_bodies_df)

,orbiting_body
0,Earth


In [ ]:
!pip install streamlit pyngrok pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import sqlite3
import pandas as pd
from datetime import date

st.set_page_config(layout="wide")

conn = sqlite3.connect("asteroids.db", check_same_thread=False)

query_options = {
    "1. Asteroid Approach Count": """
        SELECT neo_reference_id, COUNT(*) AS approach_count
        FROM close_approach GROUP BY neo_reference_id
    """,
    "2. Average Velocity per Asteroid": """
        SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
        FROM close_approach GROUP BY neo_reference_id
    """,
    "3. Top 10 Fastest Asteroids": """
        SELECT neo_reference_id, MAX(relative_velocity_kmph) AS max_velocity
        FROM close_approach GROUP BY neo_reference_id
        ORDER BY max_velocity DESC LIMIT 10
    """,
    "4. Hazardous Asteroids (>3 Approaches)": """
        SELECT a.name, ca.neo_reference_id, COUNT(*) AS approaches
        FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id
        WHERE a.is_potentially_hazardous = 1 GROUP BY ca.neo_reference_id HAVING approaches > 3
    """,
    "5. Month with Most Approaches": """
        SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS count
        FROM close_approach GROUP BY month ORDER BY count DESC LIMIT 1
    """,
    "6. Fastest Ever Asteroid": """
        SELECT neo_reference_id, MAX(relative_velocity_kmph) AS max_speed
        FROM close_approach
    """,
    "7. Asteroids by Max Estimated Diameter": """
        SELECT name, estimated_max_diameter
        FROM asteroids ORDER BY estimated_max_diameter DESC
    """,
    "8. Asteroids Getting Closer Over Time": """
        SELECT neo_reference_id, close_approach_date, miss_distance_km
        FROM close_approach ORDER BY neo_reference_id, close_approach_date
    """,
    "9. Closest Approach per Asteroid": """
        SELECT a.name, ca.close_approach_date, MIN(ca.miss_distance_km) AS closest_distance
        FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id
        GROUP BY a.name
    """,
    "10. Velocity > 50,000 km/h": """
        SELECT DISTINCT a.name, ca.relative_velocity_kmph
        FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id
        WHERE ca.relative_velocity_kmph > 50000
    """,
    "11. Approaches Per Month": """
        SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS count
        FROM close_approach GROUP BY month
    """,
    "12. Brightest Asteroid (Lowest Magnitude)": """
        SELECT name, MIN(absolute_magnitude_h) AS brightest
        FROM asteroids
    """,
    "13. Hazardous vs Non-Hazardous": """
        SELECT is_potentially_hazardous, COUNT(*) AS count
        FROM asteroids GROUP BY is_potentially_hazardous
    """,
    "14. Closer than the Moon (<1 LD)": """
        SELECT a.name, ca.close_approach_date, ca.miss_distance_lunar
        FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id
        WHERE ca.miss_distance_lunar < 1
    """,
    "15. Within 0.05 AU": """
        SELECT a.name, ca.close_approach_date, ca.astronomical
        FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id
        WHERE ca.astronomical < 0.05
    """
}

st.sidebar.title("📊 Asteroid Analysis")
selected_query = st.sidebar.selectbox("Choose a predefined query:", list(query_options.keys()))

if selected_query:
    st.subheader(f"Result: {selected_query}")
    df = pd.read_sql_query(query_options[selected_query], conn)
    st.dataframe(df, use_container_width=True)

st.markdown("---")
st.header("🔍 Filter Asteroid Approaches")

from datetime import date
col1, col2 = st.columns(2)

with col1:
    start_date = st.date_input("Start Date", value=date(2024, 1, 1))
    end_date = st.date_input("End Date", value=date(2025, 1, 1))
    au = st.slider("Astronomical Distance (AU)", 0.0, 1.0, (0.0, 0.1), step=0.01)
    velocity = st.slider("Relative Velocity (km/h)", 0, 100000, (0, 50000), step=1000)

with col2:
    ld = st.slider("Lunar Distance", 0.0, 50.0, (0.0, 10.0), step=1.0)
    min_diam = st.slider("Min Estimated Diameter (km)", 0.0, 10.0, (0.0, 1.0))
    max_diam = st.slider("Max Estimated Diameter (km)", 0.0, 20.0, (0.0, 5.0))
    hazard = st.selectbox("Hazardous?", ("All", "Yes", "No"))

where_clause = f"""
WHERE ca.close_approach_date BETWEEN '{start_date}' AND '{end_date}'
AND ca.astronomical BETWEEN {au[0]} AND {au[1]}
AND ca.relative_velocity_kmph BETWEEN {velocity[0]} AND {velocity[1]}
AND ca.miss_distance_lunar BETWEEN {ld[0]} AND {ld[1]}
AND a.estimated_min_diameter >= {min_diam[0]} AND a.estimated_max_diameter <= {max_diam[1]}
"""

if hazard == "Yes":
    where_clause += " AND a.is_potentially_hazardous = 1"
elif hazard == "No":
    where_clause += " AND a.is_potentially_hazardous = 0"

dynamic_query = f"""
SELECT a.name, ca.close_approach_date, ca.relative_velocity_kmph,
       ca.astronomical, ca.miss_distance_lunar, a.estimated_max_diameter,
       a.is_potentially_hazardous
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
{where_clause}
ORDER BY ca.close_approach_date ASC
"""

st.subheader("Filtered Asteroid Data")
filtered_df = pd.read_sql_query(dynamic_query, conn)
st.dataframe(filtered_df, use_container_width=True)

Writing app.py


In [ ]:
!pip install streamlit_extras

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.8/680.8 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.4/635.4 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
!rm -f /root/.ngrok2/ngrok.yml

In [ ]:
!ngrok config add-authtoken 2zHq04sp9S3sebon1cCUN5CYgJh_24d84TEea9Z1p2hw4KiWU

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import time

!streamlit run app.py &>/content/logs.txt &


time.sleep(5)

public_url = ngrok.connect(8501)
print(f"✅ Streamlit is running at: {public_url}")


✅ Streamlit is running at: NgrokTunnel: "https://9475-34-135-158-244.ngrok-free.app" -> "http://localhost:8501"
